In [1]:
# # demo_conversion.py
# import torch
# import pickle
# import numpy as np
# from hparams import hparams
# from utils import pad_seq_to_2, quantize_f0_numpy
# from model import Generator_3 as Generator
# from model import Generator_6 as F0_Converter

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# # -------------------------
# # Load Models & Checkpoints
# # -------------------------
# G = Generator(hparams).eval().to(device)
# g_checkpoint = torch.load('assets/100000-G.ckpt', map_location=lambda storage, loc: storage)
# G.load_state_dict(g_checkpoint['model'], strict=False)

# P = F0_Converter(hparams).eval().to(device)
# p_checkpoint = torch.load('assets/100000-P.ckpt', map_location=lambda storage, loc: storage)
# P.load_state_dict(p_checkpoint['model'])

# # -------------------------
# # Load Demo Metadata
# # -------------------------
# metadata = pickle.load(open('assets/demo.pkl', "rb"))

# # -------------------------
# # Process Source Utterance (sbmt_i)
# # -------------------------
# sbmt_i = metadata[0]
# # Load source speaker embedding and ensure it is 2D: [batch, d_emb]
# emb_org = torch.from_numpy(sbmt_i[1]).to(device)
# if emb_org.dim() == 1:
#     emb_org = emb_org.unsqueeze(0)

# x_org, f0_org, len_org, uid_org = sbmt_i[2]
# uttr_org_pad, _ = pad_seq_to_2(x_org[np.newaxis, :, :], 192)
# uttr_org_pad = torch.from_numpy(uttr_org_pad).to(device)

# # Process F0 for source: if raw (1D), pad & quantize; otherwise, adjust dimensions.
# if f0_org.ndim == 1:
#     f0_org_pad = np.pad(f0_org, (0, 192 - len_org), 'constant', constant_values=(0, 0))
#     f0_org_quantized = quantize_f0_numpy(f0_org_pad)[0]
# else:
#     if f0_org.shape[0] < 192:
#         f0_org_quantized = np.pad(f0_org, ((0, 192 - f0_org.shape[0]), (0, 0)),
#                                    'constant', constant_values=(0, 0))
#     else:
#         f0_org_quantized = f0_org[:192, :]
# f0_org_onehot = f0_org_quantized[np.newaxis, :, :]
# f0_org_onehot = torch.from_numpy(f0_org_onehot).to(device)
# uttr_f0_org = torch.cat((uttr_org_pad, f0_org_onehot), dim=-1)

# # -------------------------
# # Process Target Utterance (sbmt_j)
# # -------------------------
# sbmt_j = metadata[1]
# # Load target speaker embedding and ensure it is 2D: [batch, d_emb]
# emb_trg = torch.from_numpy(sbmt_j[1]).to(device)
# if emb_trg.dim() == 1:
#     emb_trg = emb_trg.unsqueeze(0)

# x_trg, f0_trg, len_trg, uid_trg = sbmt_j[2]
# uttr_trg_pad, _ = pad_seq_to_2(x_trg[np.newaxis, :, :], 192)
# uttr_trg_pad = torch.from_numpy(uttr_trg_pad).to(device)

# if f0_trg.ndim == 1:
#     f0_trg_pad = np.pad(f0_trg, (0, 192 - len_trg), 'constant', constant_values=(0, 0))
#     f0_trg_quantized = quantize_f0_numpy(f0_trg_pad)[0]
# else:
#     if f0_trg.shape[0] < 192:
#         f0_trg_quantized = np.pad(f0_trg, ((0, 192 - f0_trg.shape[0]), (0, 0)),
#                                    'constant', constant_values=(0, 0))
#     else:
#         f0_trg_quantized = f0_trg[:192, :]
# f0_trg_onehot = f0_trg_quantized[np.newaxis, :, :]
# f0_trg_onehot = torch.from_numpy(f0_trg_onehot).to(device)

# # -------------------------
# # Run F0 Converter to Get Target F0 Features
# # -------------------------
# with torch.no_grad():
#     f0_pred = P(uttr_org_pad, f0_trg_onehot)[0]
#     f0_pred_quantized = f0_pred.argmax(dim=-1).squeeze(0)
#     f0_con_onehot = torch.zeros((1, 192, 257), device=device)
#     f0_con_onehot[0, torch.arange(192), f0_pred_quantized] = 1
# uttr_f0_trg = torch.cat((uttr_org_pad, f0_con_onehot), dim=-1)

# # -------------------------
# # Run Generator Under Different Conditions
# # -------------------------
# conditions = ['FU', 'F', 'U', 'R', 'RF', 'RU', 'RFU']
# spect_vc = []
# with torch.no_grad():
#     for condition in conditions:
#         if condition == 'R':
#             x_identic_val, var, mu = G(uttr_f0_org, uttr_trg_pad, emb_org)
#         if condition == 'F':
#             x_identic_val, var, mu = G(uttr_f0_trg, uttr_org_pad, emb_org)
#         if condition == 'U':
#             x_identic_val, var, mu = G(uttr_f0_org, uttr_org_pad, emb_trg)
#         if condition == 'RF':
#             x_identic_val, var, mu = G(uttr_f0_trg, uttr_trg_pad, emb_org)
#         if condition == 'RU':
#             x_identic_val, var, mu = G(uttr_f0_org, uttr_trg_pad, emb_trg)
#         if condition == 'FU':
#             x_identic_val, var, mu = G(uttr_f0_trg, uttr_org_pad, emb_trg)
#         if condition == 'RFU':
#             x_identic_val, var, mu = G(uttr_f0_trg, uttr_trg_pad, emb_trg)
            
#         # Choose output length: if condition contains 'R', use len_trg; otherwise, use len_org.
#         # if 'R' in condition:
#         #     uttr_trg_out = x_identic_val[0, :len_trg, :].cpu().numpy()
#         # else:
#         #     uttr_trg_out = x_identic_val[0, :len_org, :].cpu().numpy()
            
#         uttr_trg_out = x_identic_val[0, :len_org, :].cpu().numpy()
                
#         spect_vc.append( ('{}_{}_{}_{}'.format(sbmt_i[0], sbmt_j[0], uid_org, condition), uttr_trg_out) )

# print("Generated spectrograms for conditions:", [cond for cond, _ in spect_vc])

import torch
import pickle
import numpy as np
from hparams import hparams
from utils import pad_seq_to_2, quantize_f0_numpy
from model import Generator_3 as Generator

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# -------------------------
# Load Model & Checkpoint for Generator
# -------------------------
G = Generator(hparams).eval().to(device)
g_checkpoint = torch.load('assets/800000-G-B10.ckpt', map_location=lambda storage, loc: storage)
G.load_state_dict(g_checkpoint['model'], strict=False)

# -------------------------
# Load Demo Metadata
# -------------------------
metadata = pickle.load(open('assets/demo.pkl', "rb"))

# -------------------------
# Process Source Utterance (sbmt_i)
# -------------------------
sbmt_i = metadata[0]
# Load source speaker embedding and ensure it is 2D: [batch, d_emb]
emb_org = torch.from_numpy(sbmt_i[1]).to(device)
if emb_org.dim() == 1:
    emb_org = emb_org.unsqueeze(0)

x_org, f0_org, len_org, uid_org = sbmt_i[2]
# Crop x_org to 192 frames if necessary
if x_org.shape[0] > 192:
    x_org = x_org[:192, :]
    len_org = 192
# Also crop f0_org to 192 frames if needed (for 1D f0)
if f0_org.ndim == 1 and f0_org.shape[0] > 192:
    f0_org = f0_org[:192]
uttr_org_pad, _ = pad_seq_to_2(x_org[np.newaxis, :, :], 192)
uttr_org_pad = torch.from_numpy(uttr_org_pad).to(device)

# Process F0 for source: if raw (1D), pad & quantize; otherwise, adjust dimensions.
if f0_org.ndim == 1:
    f0_org_pad = np.pad(f0_org, (0, 192 - len_org), 'constant', constant_values=(0, 0))
    f0_org_quantized = quantize_f0_numpy(f0_org_pad)[0]
else:
    if f0_org.shape[0] < 192:
        f0_org_quantized = np.pad(f0_org, ((0, 192 - f0_org.shape[0]), (0, 0)),
                                   'constant', constant_values=(0, 0))
    else:
        f0_org_quantized = f0_org[:192, :]
f0_org_onehot = f0_org_quantized[np.newaxis, :, :]
f0_org_onehot = torch.from_numpy(f0_org_onehot).to(device)
uttr_f0_org = torch.cat((uttr_org_pad, f0_org_onehot), dim=-1)

# -------------------------
# Process Target Utterance (sbmt_j)
# -------------------------
sbmt_j = metadata[1]
# Load target speaker embedding and ensure it is 2D: [batch, d_emb]
emb_trg = torch.from_numpy(sbmt_j[1]).to(device)
if emb_trg.dim() == 1:
    emb_trg = emb_trg.unsqueeze(0)

x_trg, f0_trg, len_trg, uid_trg = sbmt_j[2]
# Crop x_trg to 192 frames if necessary
if x_trg.shape[0] > 192:
    x_trg = x_trg[:192, :]
    len_trg = 192
# Also crop f0_trg to 192 frames if needed (for 1D f0)
if f0_trg.ndim == 1 and f0_trg.shape[0] > 192:
    f0_trg = f0_trg[:192]
uttr_trg_pad, _ = pad_seq_to_2(x_trg[np.newaxis, :, :], 192)
uttr_trg_pad = torch.from_numpy(uttr_trg_pad).to(device)

if f0_trg.ndim == 1:
    f0_trg_pad = np.pad(f0_trg, (0, 192 - len_trg), 'constant', constant_values=(0, 0))
    f0_trg_quantized = quantize_f0_numpy(f0_trg_pad)[0]
else:
    if f0_trg.shape[0] < 192:
        f0_trg_quantized = np.pad(f0_trg, ((0, 192 - f0_trg.shape[0]), (0, 0)),
                                   'constant', constant_values=(0, 0))
    else:
        f0_trg_quantized = f0_trg[:192, :]
f0_trg_onehot = f0_trg_quantized[np.newaxis, :, :]
f0_trg_onehot = torch.from_numpy(f0_trg_onehot).to(device)

# Instead of using the F0 converter, directly create the target F0 input:
uttr_f0_trg = torch.cat((uttr_trg_pad, f0_trg_onehot), dim=-1)

# -------------------------
# Run Generator Under Different Conditions
# -------------------------
conditions = ['R', 'F', 'U', 'FU', 'RF', 'RU', 'RFU']
spect_vc = []
with torch.no_grad():
    for condition in conditions:
        if condition == 'R':
            x_identic_val, var, mu = G(uttr_f0_org, uttr_trg_pad, emb_org)
        if condition == 'F':
            x_identic_val, var, mu = G(uttr_f0_trg, uttr_org_pad, emb_org)
        if condition == 'U':
            x_identic_val, var, mu = G(uttr_f0_org, uttr_org_pad, emb_trg)
        if condition == 'RF':
            x_identic_val, var, mu = G(uttr_f0_trg, uttr_trg_pad, emb_org)
        if condition == 'RU':
            x_identic_val, var, mu = G(uttr_f0_org, uttr_trg_pad, emb_trg)
        if condition == 'FU':
            x_identic_val, var, mu = G(uttr_f0_trg, uttr_org_pad, emb_trg)
        if condition == 'RFU':
            x_identic_val, var, mu = G(uttr_f0_trg, uttr_trg_pad, emb_trg)
            
        if 'R' in condition:
            uttr_trg_out = x_identic_val[0, :len_trg, :].cpu().numpy()
        else:
            uttr_trg_out = x_identic_val[0, :len_org, :].cpu().numpy()
                
        spect_vc.append( ('{}_{}_{}_{}'.format(sbmt_i[0], sbmt_j[0], uid_org, condition), uttr_trg_out) )

print("Generated spectrograms for conditions:", [cond for cond, _ in spect_vc])


Generated spectrograms for conditions: ['p558_p547_p558_001.npy_R', 'p558_p547_p558_001.npy_F', 'p558_p547_p558_001.npy_U', 'p558_p547_p558_001.npy_FU', 'p558_p547_p558_001.npy_RF', 'p558_p547_p558_001.npy_RU', 'p558_p547_p558_001.npy_RFU']


In [2]:
# spectrogram to waveform
import torch
import soundfile
import pickle
import os
from synthesis import build_model
from synthesis import wavegen

if not os.path.exists('results'):
    os.makedirs('results')

model = build_model().to(device)
checkpoint = torch.load("assets/checkpoint_step001000000_ema.pth", map_location=torch.device(device))
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)   
    soundfile.write('results/'+name+'.wav', waveform, samplerate=16000)

/usr/lib/python3/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


p558_p547_p558_001.npy_R


100%|████████████████████████████████████| 30976/30976 [02:49<00:00, 182.57it/s]


p558_p547_p558_001.npy_F


100%|████████████████████████████████████| 34560/34560 [03:09<00:00, 182.01it/s]


...

100%|████████████████████████████████████| 30976/30976 [02:49<00:00, 182.39it/s]


p558_p547_p558_001.npy_RU


100%|████████████████████████████████████| 30976/30976 [02:49<00:00, 182.69it/s]


p558_p547_p558_001.npy_RFU


100%|████████████████████████████████████| 30976/30976 [02:49<00:00, 182.59it/s]
